In [1]:
import org.apache.spark.sql._
import org.apache.spark.ml._
import java.io._
import play.api.libs.json._
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._

In [2]:
val vehicleUsers = spark.read.json("/labs/laba02/autousers.json").toDF()

Waiting for a Spark session to start...

vehicleUsers = [autousers: array<string>]


[autousers: array<string>]

In [3]:
val logs = spark.read.option("delimiter", "\t").textFile("/labs/laba02/logs")

logs = [value: string]


[value: string]

In [4]:
logs.show(3, truncate=false)

+-----------------------------------------------------------------------------------------------------+
|value                                                                                                |
+-----------------------------------------------------------------------------------------------------+
|258131083251	1423724291.637	http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251	1423724290.637                                                                          |
|182984926014	1423724291.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
+-----------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [5]:
// split df to array rows

val col_ = split(col("value"), "\t").alias("value")
val logs_splited = logs.select(col_)
logs_splited.show(3, truncate=false)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|[258131083251, 1423724291.637, http%3A%2F%2Fwww.zakon.kz%2F]                                             |
|[258131083251, 1423724290.637]                                                                           |
|[182984926014, 1423724291.666, http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5]|
+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows



col_ = split(value, 	) AS `value`
logs_splited = [value: array<string>]


[value: array<string>]

In [17]:
// selecting columns by index of arrays

val logs_df = logs_splited.withColumn("UID", $"value"(0))
                    .withColumn("Timestamp", $"value"(1))
                    .withColumn("URL", $"value"(2))
                    .withColumn("Timestamp", to_timestamp(from_unixtime($"Timestamp")))
                    .select("UID","Timestamp", "URL")
                    .filter($"URL".like("http%"))


logs_df = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [16]:
logs_df.show(5, truncate=false)

+------------+-------------------+-------------------------------------------------------------------------+
|UID         |Timestamp          |URL                                                                      |
+------------+-------------------+-------------------------------------------------------------------------+
|258131083251|2015-02-12 09:58:11|http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251|2015-02-12 09:58:10|null                                                                     |
|182984926014|2015-02-12 09:58:11|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
|182984926014|2015-02-12 09:58:10|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D4|
|289380960531|2015-02-12 09:58:11|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fforumdisplay.php%3Ff%3D128            |
+------------+-------------------+-------------------------------------------------------------------------+
only showing top 5 

In [33]:
// extracting neat URL

val substrDel = substring_index($"URL", ".", 1)
val logs_df_clean = logs_df.withColumn("URL", substring_index(substring(regexp_replace($"URL",substrDel , lit("")), 2, 999), "%", 1))

logs_df_clean.show(numRows=5, truncate=200 /*, vertical=true*/)

+------------+-------------------+----------+
|         UID|          Timestamp|       URL|
+------------+-------------------+----------+
|258131083251|2015-02-12 09:58:11|  zakon.kz|
|182984926014|2015-02-12 09:58:11|bmwclub.ru|
|182984926014|2015-02-12 09:58:10|bmwclub.ru|
|289380960531|2015-02-12 09:58:11|bmwclub.ru|
|289380960531|2015-02-12 09:58:10|bmwclub.ru|
+------------+-------------------+----------+
only showing top 5 rows



substrDel = substring_index(URL, ., 1)
logs_df_clean = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [34]:
//logs_df.withColumn("URL", trim(LEADING, lit("http"),  $"URL")).show(3)

Name: Syntax Error.
Message: 
StackTrace: 

In [36]:
logs_df_clean.printSchema()

root
 |-- UID: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- URL: string (nullable = true)



In [37]:
val vehicleUsers_df = vehicleUsers.select(explode($"autousers").alias("autousers"))
vehicleUsers_df.show(3)

+------------+
|   autousers|
+------------+
|100341861572|
|100473724387|
|100528753939|
+------------+
only showing top 3 rows



vehicleUsers_df = [autousers: string]


[autousers: string]

In [38]:
logs_df_clean.select($"UID").distinct.count()

100000

In [40]:
vehicleUsers_df.select("autousers").distinct().count()

5209